In [ ]:
# gmail 서버를 활용하여 자동으로 email 발송하기

# Step 1. 필요한 모듈을 import 합니다
import email,smtplib,os
import openpyxl , sys
from io import StringIO
import time
import random

# 이메일의 제목과 본문과 첨부파일을 위한 모듈
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email import encoders
from email.mime.base import MIMEBase
import email.mime.application

# Step 2. 필요한 gmail 정보를 입력합니다.
gid = input("G-Mail 로그인 계정을 입력하세요: ")
gpw = '이곳에 gmail의 Mail App 암호를 쓰세요'

s_time = time.time( )

# Step 3. 첨부 파일 정보를 지정합니다
#file_name = input("메일에 첨부할 파일명: ")
txt_files = "c:\\temp\\mail_test\\stop_words.txt"
img_files = "c:\\temp\\mail_test\\wordcloud.png"

# Step 4. 엑셀 파일에서 목록 불러와서 자동으로 메일을 발송합니다
wb = openpyxl.load_workbook("c:\\temp\\email_list.xlsx")
sheet = wb["Sheet1"]

member = { }

for i in range(2, sheet.max_row + 1) :
    name = sheet.cell(row=i,column=1).value
    email = sheet.cell(row=i,column=2).value
    member[name]=email

smtp_1 = smtplib.SMTP_SSL('smtp.gmail.com',465)
smtp_1.ehlo( )
smtp_1.login(gid,gpw)

count = 0
complete_list= [ ]
fail_list = [ ]

for name, email in member.items( ):

    #보낼 메일 내용을 직접 입력하실 경우 아래 줄의 주석을 삭제하고 사용하세요
    #msg=MIMEText("%s님께 \n지난달 사용요금을 청구합니다" %name)

    #만약 보낼 내용을 특정 파일에서 읽어와서 사용할 경우는 아래와 같이 코딩하세요.
    #기본값은 파일에서 메일 내용을 읽어와서 보내도록 되어 있습니다.
    
    with open("c:\\temp\\mail_test\\ment.txt") as ment:
        msgtxt = MIMEText(ment.read() %name)
     
    #txt첨부파일 등록
    filename=txt_files
    part = MIMEBase('application', "octet-stream")
    part.set_payload(open(filename, "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(filename))
    msg = MIMEMultipart()
    msg.attach(part)
   
    #image 첨부파일 등록
    filename2=img_files
    part2 = MIMEBase('application', "octet-stream")
    part2.set_payload(open(filename2, "rb").read())
    encoders.encode_base64(part2)
    part2.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(filename2))

    #메일 보낼 내용 설정 -텍스트 첨부파일과 이미지 첨부파일과 메시지 본문 설정과 헤드 설정
    msg = MIMEMultipart()
    msg.attach(part)
    msg.attach(part2)
    msg.attach(msgtxt)
    msg["Subject"] = Header(s="가치랩스에서 %s 님께 알려 드립니다."%name , charset="utf-8")   
    sendmailStatus=smtp_1.sendmail('seojinsu@gmail.com',email,msg.as_string()) 
    time.sleep(random.randrange(2,6))
    
    n = time.localtime()
    s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
    
    print("=" *60)
    print("%s 주소로 메일 발송 시간은 %s 입니다" %(email,s))
    print("%s 주소로 메일 발송완료------"%email)

    complete_list.append(email)
    
    count += 1
    
    if sendmailStatus != {}:
        print("%s 주소 발송 오류: %s" %(email,sendemailStatus))
       
e_time = time.time( )
t_time = e_time - s_time

#발송 완료 명단을 파일에 저장합니다.
com_list = "c:\\temp\\email_com_list.txt"

f = open(com_list, 'a',encoding='UTF-8')
f.write(str(complete_list))
f.close( )

print("=" *60)
print("총 소요시간:  %s 초 입니다~" %t_time)
print("총 %s 건 발송 완료했습니다~!" %count)
print("발송 완료 명단은 %s 에 저장되어 있습니다" %com_list)

smtp_1.quit( )